Будем осуществлять работу с непростым набором данных “horse_data.csv” о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [32]:
import pandas as pd
import numpy as np

In [97]:
%pip install phik
import phik
from phik.report import plot_correlation_matrix

Note: you may need to restart the kernel to use updated packages.


In [235]:
horse_data = pd.read_csv('/Users/sofagusina/Desktop/программирование/machine_learning/статистика/hw2/Файлы_для_ДЗ/horse_data.csv')
"""Преообразование дата-фрейма к новому, в котором будет 8 конкретных столбцов:surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome. """
columns_horse_data = horse_data.iloc[:, np.r_[0:8, 11, 23]]
columns_horse_data = columns_horse_data.replace('?', np.NaN).apply(pd.to_numeric)
"""Посмотрим основные статистические характеристики и выбросы"""
columns_horse_data.describe() 
for column in columns_horse_data.columns:
    col_data = columns_horse_data[column]
    iqr = col_data.quantile(0.75) - col_data.quantile(0.25)
    lower_bound = col_data.quantile(0.25) - 1.5 * iqr
    upper_bound = col_data.quantile(0.75) + 1.5 * iqr
    outliers = []
    for el in col_data:
        if el < lower_bound or el > upper_bound:
            outliers.append(el)
    print(f"Outliers in column {column}:", outliers)

"""Просмотр количества пропусков."""
columns_horse_data.info()
"""Пропуски есть в столбцах: surgery, rectal temperature, pulse, respiratory rate,temperature of extremities, peripheral pulse,pain"""
"""Посмотрим корреляцию между данными, чтобы понять, каким образом нужно работать с пропусками"""
phik_overwiew = columns_horse_data.phik_matrix()
plot_correlation_matrix(phik_overwiew.values, x_labels = phik_overwiew.columns, y_labels = phik_overwiew.index,figsize =(10,8))
"""Вывод: 
наличие операции - зависит только в меньшей степени от исхода
температура ректальная -  пульс и частота дыхания,
пульс - частота дыхания и температура
частота дыхания - пульс, ректальная температура
температура конечностей - боль и периферийный пульс
периферийный пульс - боль
боль -  температура конечностей и периферийный пульс
"""

""" Необходимо проверить, как повлияет замена/удаление пропуска на основыне стат.характеристики данных (среднее, медиана и мода)"""

"""В столбце с ректальной температурой заменим пропуски средним значением и проверим корреляцию между температурой и пульсом и температурой и частотой дыхания"""
columns_horse_data['38.50'] = columns_horse_data['38.50'].fillna(columns_horse_data['38.50'].mean())
"""Корреляция изменилась незначительно, значит, можно заполнить пропуски средним значением"""

"""Работа с пропусками в столбце пульс.
 При удалении строчек с пропусками меняются статистические показатели 
 - температуры изменяются несущественно.
 - частоты дыхания - медиана
 - температура конечностей - несущественно
 - пер/пульс - медиана
 - боль - несущественно
 - исход - несущественно
 Вывод: заменим медианой, чтобы сохранить стат показатели других данных, и проверим корреляцию между частотой дыхания и температурой
 """

columns_horse_data['66'] = columns_horse_data['66'].fillna(columns_horse_data['66'].median())
"""Корреляция изменяется несущественно, статистические характеристики частоты дыхания и пер/пульса практически не меняются"""

"""Работа с пропусками в столбце частота дыхания.
 При удалении строчек с пропусками меняются статистические показатели 
 - температуры изменяются несущественно.
 - пульс - среднее значение
 - температура конечностей - несущественно
 - пер/пульс - медиана
 - боль - несущественно
 - исход - несущественно
 Вывод: заменим средним, чтобы сохранить стат показатели других данных, и проверим корреляцию между пульсом и ректальной температурой
 """
columns_horse_data['28'] = columns_horse_data['28'].fillna(columns_horse_data['28'].mean())
"""Корреляция изменяется несущественно, статистичексие характеристики пульса и пер/пульса практически не меняются"""

"""Работа с пропусками в столбце температура конечностей.
 При удалении строчек с пропусками меняются статистические показатели 
 - температуры изменяются несущественно.
 - пульс - медиана
 - частота дыхания - несущественно
 - пер/пульс - мода
 - боль - несущественно
 - исход - несущественно
 Вывод: заменим медианой, чтобы сохранить стат показатели других данных, и проверим корреляцию между болью и периферийным пульсом
 """
columns_horse_data['3'] = columns_horse_data['3'].fillna(columns_horse_data['3'].median())
"""Корреляция изменяется несущественно, статистичексие характеристики пульса и пер/пульса практически не меняются"""

"""Работа с пропусками в столбце периферийный пульс.
 При удалении строчек с пропусками меняются статистические показатели 
 -температуры изменяются несущественно.
 - пульс - несущественно
 - частота дыхания - несущественно
 - температура конечностей - несущественно
 - боль - несущественно
 - исход - несущественно
 Вывод: пропуски можно удалить, корреляция с болью изменяется несущественно
 """
columns_horse_data.dropna(subset=['3.1'])

"""Работа с пропусками в столбце боль.
 При удалении строчек с пропусками меняются статистические показатели 
 -температуры изменяются несущественно.
 - пульс - очень сильно медиана
 - частота дыхания - несущественно
 - температура конечностей - несущественн
 - пер/пульс - мода
 - исход - несущественно
 Вывод: заменим медианой, чтобы сохранить стат показатели других данных, и проверим корреляцию между температурой конечностей и периферийным пульсом
 """

columns_horse_data['4'] = columns_horse_data['4'].fillna(columns_horse_data['4'].median())
"""Корреляция изменяется несущественно, статистичексие характеристики пульса и пер/пульса практически не меняются"""

"""Ниже представлены закомментированный примеры расчета стат.показателей и сравнение их после удаление пропусков в определенном столбце"""
 # print(columns_horse_data['3.1'].mean())
 # print(columns_horse_data['3.1'].mode()[0])
 # print(columns_horse_data['3.1'].median())

 # print(columns_horse_data.dropna(subset=['4'])['2.3'].mean())
 # print(columns_horse_data.dropna(subset=['4'])['2.3'].mode()[0])
 # print(columns_horse_data.dropna(subset=['4'])['2.3'].median())
"""Матрица корреляции служила для проверки корреляции данных после работы с пропусками"""
phik_overwiew = columns_horse_data.phik_matrix()
plot_correlation_matrix(phik_overwiew.values, x_labels = phik_overwiew.columns, y_labels = phik_overwiew.index,figsize =(10,8))

"""Если указано про несущественное изменение (или что практически не изменяется), то имеется в виду, что изменение показателей лежит в пределах 1 единицы"""



Outliers in column 2: []
Outliers in column 1: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Outliers in column 530101: [5290409, 5299629, 5291329, 5301219, 5290481, 5279442, 5277409, 5288249, 5282839, 5299253, 5292929, 5290759, 5279822, 5292489, 5297379, 5290482, 5262541, 5287179, 518476, 5279822, 5278331, 5299603, 5287279, 5294369, 5279441, 5289419, 5262543, 5281091, 5291409, 5291719, 5294539, 5305629, 5297159, 5283431, 5275212, 5305129]
Outliers in column 38.50: [39.9, 35.4, 40.3, 39.7, 36.4, 40.3, 39.6, 36.5, 36.0, 36.1, 36.6, 40.8, 40.0, 36.5]
Outliers in column 66: [164.0, 150.0, 160.0, 184.0, 150.0]
Outliers in column 28: [84.0, 96.0, 72.0, 80.0, 80.0, 68.0, 96.0, 66.0, 68.0, 90.0, 80.0, 70.0, 88.0, 84.0, 68.0, 90.0, 70.0]
Outliers in column 3: []
Outliers in column 3.1: []
Outliers in column 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

'Пропуски есть в столбцах: surgery,'